# Capstone Project: IMDB User Reviews Rating predictor
## Hung Pham

## Part 1: Introduction and Data Scraping

### 1.1. Problem statement

Sentiment analysis is one of the earliest problem come to mind when dealing with NLP. With the world moving to a new digital age with almost anything can be bought and sold on the internet, knowing if an review is positive or negative toward the product is crucial for any business owner. These review may contains within it the hint to improve a product or avoid the drawback.


The most common practical application for sentiment analysis is with review. Normally, the data analyst practitioner will use the rating score to separate all the review into good and bad review, and just perform the analysis with them. The threshold to decide good from bad review is differ case by case, but most of the time, it would be enough to learn what make a product good or bad.

But if we want to make it further, say if we have a review and instead of knowing if the review is positive or negative, we want to actually predict the score that reviewer would give to this product. That isn't an easy task, even for human. It's even harder than a normal multi classes classification, as the classes here is closely linked with each other in a hierarchy order, instead of completely distinguishable classes (think about trying to predict if it is a 5 or 6 score versus predict if it is a cat or a dog.

With all the tools that we have in our arsenal, I decided to push it further and see how far we can go in this challenge.

### 1.2. Data collecting

#### 1.2.1. Planning

As a movies fan, I wanted to use IMDB users review database with this project. To my surprise, there aren't any dataset for it available in the popular data set collection website (Kaggle, Google analytic...).   
So I decided to scrap the data directly from IDMB.

The structure of IMDB is rather straight forward, each movies will have its own page contains all the information of the movies. Our main interest is the user review page.

Click on the `USER REVIEWS` link, we will be directed to the corresponding page of the movie's user reviews.

After review functionalities and limit of several data scraping tools, I decided to use a combination of **Selenium** and **BeautifulSoup** due to their flexibility and beginner friendliness. The drawback is that both are quite slow.  
The detail step by step process should look like this:  
- The only input we need is the `Top100 movies` website, we will use Selenium to grab all the hyperlink of all the individual movies and store them into `movies link` list.  
- Use BS4 to go to each movie website to grab it user review link, store them into `user_review_links` list.
- Go to each link in `user_review_links`, as IMDB's USER REVIEW pages are defaulted to show only 25 reviews each page and we will have to click on `load more` button to expand it. We can use Selenium to do this task for us. 
- After expanding the page, we can extract the reviews into their respected lists: `Rating`, `User_name`, `Review_text`, `Review_title`, `Review_date`. We can then create a dataframe for each movie and out them into a csv file for later use.  
- Combine all the individual csv files back into our final dataframe.

Planning is done, let's get started!!!

#### 1.2.2. Scraping data

First step, we will need to import all the necessary libraries:

In [1]:
import pandas as pd #Using panda to create our dataframe
# Import Selenium and its sub libraries
import selenium 
from selenium import webdriver
# Import BS4
import requests #needed to load the page for BS4
from bs4 import BeautifulSoup

As we chose Chrome as our main web browser, we will need to download Chrome driver and tell Selenium where to find it:

In [2]:
PATH = r"C:\Users\ngoch\Dropbox\Data science\chromedriver.exe" #path to the webdriver file

Then we can set up our `top100` webpage, it will be the only input we need to feed in the script

In [3]:
def get_review(url, folder_name):
    '''
    Get the review from input as url for IMDB movies list.
    The function takes 2 input the url of the movies and the name of the folder to store the data
    For each folder, the function will grab the review for each movies and store into respective file.
    '''


    driver = webdriver.Chrome(PATH) #tell selenium to use Chrome and find the webdriver file in this location
    driver.get(url) #tell Selenium to open the webpage
    driver.implicitly_wait(1) # tell the webdriver to wait for 1 seconds for the page to load

    #After the webpage opened, we can extract the title, hyperlink, year of each movies
    #Set initial empty list for each element:
    title = []
    link = []
    year = []

    #Grab the block of each individual movie
    block = driver.find_elements_by_class_name('lister-item')
    #Set up for loop to run through all 50 movies in the first page
    for i in range(0,50):
        try:
            #Extracting title
            ftitle = block[i].find_element_by_class_name('lister-item-header').text

            #The extracted title has extra elements, so we will have to do some cleaning
            #Remove the order in front of the title
            forder = block[i].find_element_by_class_name('lister-item-index').text
            #Extract the year last 6 letter of the title
            fyear = ftitle[-6:]
            #Drop the order, year and only keep the movie's name
            ftitle = ftitle.replace(forder+' ', '')[:-7 ]
            #Then extract the link with cleaned title
            flink = block[i].find_element_by_link_text(ftitle).get_attribute('href')

            #Add item to the respective lists
            title.append(ftitle)
            year.append(fyear)
            link.append(flink)
        except:
            continue

    # After that, we can use BeautifulSoup to extract the user reviews link 
    #Set an empty list to store user review link
    user_review_links = []
    for url in link:
        url = url
        #setup user agent for BS4, except some rare case, it would be the same for most browser 
        user_agent = {'User-agent': 'Mozilla/5.0'}
        #Use request.get to load the whole page
        response = requests.get(url, headers = user_agent)
        #Parse the request object to BS4 to transform it into html structure
        soup = BeautifulSoup(response.text, 'html.parser')
        #Find the link marked by the USER REVIEWS link text.
        review_link = 'https://www.imdb.com'+soup.find('a', text = 'USER REVIEWS').get('href')
        #Append the newly grabed link into its list
        user_review_links.append(review_link)

    #Then create the first data frame to summarize our data at this point:
    #Create dictionary for data and columns' name
    top_50_data = {'Movie_name': title, 
            'Year': year, 
            'link': link,
            'user_review' : user_review_links,
            }
    top50 = pd.DataFrame(data = top_50_data) #create dataframe
    driver.quit() #tell Selenium to close the webpage

    # Step 2, we will grab the data from each user review page
    # Use Selenium to go to each user review page
    for i in range(len(top50['user_review'])): 
        driver = webdriver.Chrome(PATH)
        driver.get(top50['user_review'][i])
        driver.implicitly_wait(1) # tell the webdriver to wait for 1 seconds for the page to load to prevent blocked by anti spam software


        # Set up action to click on 'load more' button
        # note that each page on imdb has 25 reviews
        page = 1 #Set initial variable for while loop
        #We want at least 1000 review, so get 50 at a safe number
        while page<50:  
            try:
                #find the load more button on the webpage
                load_more = driver.find_element_by_id('load-more-trigger')
                #click on that button
                load_more.click()
                page+=1 #move on to next loadmore button
            except:
                #If couldnt find any button to click, stop
                break
        # After fully expand the page, we will grab data from whole website
        review = driver.find_elements_by_class_name('review-container')
        #Set list for each element:
        title = []
        content = []
        rating = []
        date = []
        user_name = []
        #run for loop to get 
        for n in range(0,1100):
            try:
                #Some reviewers only give review text or rating without the other, 
                #so we use try/except here to make sure each block of content must has all the element before append them to the list

                #Check if each review has all the elements
                ftitle = review[n].find_element_by_class_name('title').text
                #For the review content, some of them are hidden as spoiler, 
                #so we use the attribute 'textContent' here after extracting the 'content' tag
                fcontent = review[n].find_element_by_class_name('content').get_attribute("textContent").strip()
                frating = review[n].find_element_by_class_name('rating-other-user-rating').text
                fdate = review[n].find_element_by_class_name('review-date').text
                fname = review[n].find_element_by_class_name('display-name-link').text


                #Then add them to the respective list
                title.append(ftitle)
                content.append(fcontent)
                rating.append(frating)
                date.append(fdate)
                user_name.append(fname)
            except:
                continue
        #Build data dictionary for dataframe
        data = {'User_name': user_name, 
            'Review title': title, 
            'Review Rating': rating,
            'Review date' : date,
            'Review_body' : content
           }
        #Build dataframe for each movie to export
        review = pd.DataFrame(data = data)
        movie = top50['Movie_name'][i] #grab the movie name from the top50 list    
        review['Movie_name'] = movie #create new column with the same movie name column    
        review.to_csv(f'data/{folder_name}/{i+1}.csv') #store them into individual file for each movies, so we can combine or check them later
        driver.quit()

Then we can save the newly created raw dataframe into a csv file for future use

The whole process gave us around 50k reviews for top 50 highest rating movies on IMDB, but as these are the best of the best, I expected that the rating will be heavily imblanced as most of them will be 9/10 score. To combat that, I also repeat the process, but with top 50 lowest rating movies. 
So the raw data after combined will have reviews from both good and bad movies.

The 3 cells below are the code to grab the data and save them into respective folder inside main `Data` folder.  
I wont pre-run these cells again as all they do is write the `csv` file and give no output.

In [ ]:
top50_raw_link = 'https://www.imdb.com/search/title/?groups=top_100'
get_review(top50_raw_link, 'top50_raw')

In [ ]:
bot50_2_link = 'https://www.imdb.com/search/title/?groups=bottom_100&start=51&ref_=adv_nxt'
get_review(bot50_2_link, 'bottom50_2_raw')

In [ ]:
bot50_1_link = 'https://www.imdb.com/search/title/?groups=bottom_100&ref_=adv_prv'
get_review(bot50_1_link, 'bottom50_1_raw')

After getting our data. Let's combine them together and have some EDA in Part2_Cleaning_and_EDA

In [ ]:
df_bad_raw.to_csv('data/bad_review_half.csv')

In [ ]:
df_bad_raw1 = df_bad_raw

In [ ]:
df_bad_raw2 = pd.read_csv('data/review_bad1.csv')

In [ ]:
df_bad_raw = pd.concat([df_bad_raw1,df_bad_raw2])

In [ ]:
df_bad_raw.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1, inplace = True)

In [ ]:
df_bad_raw

In [ ]:
df_bad_raw['Review_body'] = df_bad_raw['Review_body'].str.replace('\n', '') # Remove linebreak symbol
df_bad_raw['Review_body'] = df_bad_raw['Review_body'].str.replace('Was this review helpful?', '')
df_bad_raw['Review_body'] = df_bad_raw['Review_body'].str.replace('Sign in to vote. ', '')
df_bad_raw['Review_body'] = df_bad_raw['Review_body'].str.replace('Permalink', '')
df_bad_raw['Review_body'] = [re.sub(r"\s{2,}", "", x) for x in df_bad_raw['Review_body']]

In [ ]:
df_bad_raw.columns = ['User_name', 'Review_title', 'Review_Rating', 'Review_date',
       'Review_body', 'Movie_name']

In [ ]:
df_bad_raw[['Review_Rating','temp']] = df_bad_raw['Review_Rating'].str.split(pat = '/', expand = True)
df_bad_raw.drop('temp', axis=1, inplace=True)

In [ ]:
df_bad_raw['Review_date'] = pd.to_datetime(df_bad_raw['Review_date'])

In [ ]:
df_bad_raw['Review_Rating'] = df_bad_raw['Review_Rating'].astype('int8')

In [ ]:
df_bad_raw['year'] = df_bad_raw['Review_date'].dt.year

In [ ]:
df_bad_raw['month'] = df_bad_raw['Review_date'].dt.month

In [ ]:
df_bad_raw['weekday'] = df_bad_raw['Review_date'].dt.dayofweek

In [ ]:
ddf_bad_rawf = df_bad_raw.drop('Review_date', axis=1)

In [ ]:
df_bad_raw.to_csv('data/review_bad_cleaned.csv', index = False)

In [ ]:
df_bad_raw

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(df_bad_raw['Review_Rating'])